In [1]:
import pandas as pd
import re
import underthesea
import os

In [2]:
data_path = "./Dataset_For_Work"

In [3]:
def load_data(file_path, label):
    """
    Đọc dữ liệu từ file và gắn nhãn tương ứng.
    Args:
        file_path (str): Đường dẫn đến tệp văn bản.
        label (str): Nhãn cảm xúc (positive, neutral, negative).
    Returns:
        data (list): Danh sách chứa các câu và nhãn.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            sentence = line.strip()  # Xóa khoảng trắng ở đầu và cuối câu
            if sentence:  # Chỉ thêm những câu không rỗng
                data.append((sentence, label))
    return data

In [4]:
def clean_text(text):
    """
    Làm sạch văn bản bằng cách loại bỏ ký tự đặc biệt, số và khoảng trắng thừa.
    Args:
        text (str): Câu văn bản cần làm sạch.
    Returns:
        str: Văn bản sau khi làm sạch.
    """
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
    return text

In [5]:
# Đọc dữ liệu từ các tệp và gắn nhãn
positive_data = load_data(os.path.join(data_path, "SA-training_positive.txt"), "positive")
neutral_data = load_data(os.path.join(data_path, "SA-training_neutral.txt"), "neutral")
negative_data = load_data(os.path.join(data_path, "SA-training_negative.txt"), "negative")

# Kết hợp tất cả dữ liệu
all_data = positive_data + neutral_data + negative_data

# Tiền xử lý văn bản
cleaned_data = [(clean_text(sentence), label) for sentence, label in all_data]

# Chuyển dữ liệu sang DataFrame
df = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])

# Hiển thị một vài dòng đầu tiên
print(df.head())

                                            Sentence     Label
0  đang xài mx dùng bình thường ngon pin trâu mỗi...  positive
1  qủa pin ngon sạc lại được bền riêng em dùng pi...  positive
2  cũng đang xài con logitech bluetooth tầm thấp ...  positive
3  logitech pin trâu thôi rôi mua con b cui ma cu...  positive
4  em có con chuột không dây k cũng đầy đủ nút bấ...  positive
